In [21]:
import os
import pandas as pd
import csv

# Function to detect delimiter by reading the first line of the file
def detect_delimiter(txt_file_path):
    with open(txt_file_path, 'r') as file:
        first_line = file.readline()
        # Check common delimiters: comma, tab, and space
        if ',' in first_line:
            return ','
        elif '\t' in first_line:
            return '\t'
        elif ' ' in first_line:
            return ' '
        else:
            return None

# Function to convert a single .txt file to .xlsx, including header information
def convert_txt_to_xlsx(txt_file_path, output_folder):
    try:
        # Read the entire file to handle both header and data
        with open(txt_file_path, 'r') as file:
            # Assuming the header is the first few lines before the data table
            header_lines = []
            for i in range(5):  # Adjust the number of header lines accordingly
                line = file.readline().strip()
                header_lines.append(line)
        
        # Detect the delimiter
        delimiter = detect_delimiter(txt_file_path)
        if not delimiter:
            raise ValueError(f"Unknown delimiter in file: {txt_file_path}")

        # Read the actual data, skipping the header lines
        df = pd.read_csv(txt_file_path, delimiter=delimiter, skiprows=len(header_lines), engine="python", quoting=csv.QUOTE_NONE)
        
        # Prepare Excel writer
        base_name = os.path.basename(txt_file_path).replace('.txt', '.xlsx')  # Changed to .xlsx
        xlsx_file_path = os.path.join(output_folder, base_name)
        
        with pd.ExcelWriter(xlsx_file_path, engine='openpyxl') as writer:
            # Write header lines as text at the top of the Excel sheet
            header_df = pd.DataFrame(header_lines)
            header_df.to_excel(writer, sheet_name='Sheet1', index=False, header=False)

            # Append the main data underneath the header
            df.to_excel(writer, sheet_name='Sheet1', index=False, startrow=len(header_lines) + 2)
        
        print(f"Converted: {txt_file_path} to {xlsx_file_path}")

    except pd.errors.ParserError as e:
        print(f"ParserError while reading {txt_file_path}: {e}")
    except Exception as e:
        print(f"Error while converting {txt_file_path}: {e}")

# Main function to convert all .txt files in a folder to .xlsx
def batch_convert_txt_to_xlsx(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all the files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.txt'):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_txt_to_xlsx(txt_file_path, output_folder)

# Example usage:
input_folder = "/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-TXT"  # Replace with your folder path containing .txt files
output_folder = "/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel"  # Replace with the folder path where .xlsx files will be saved

batch_convert_txt_to_xlsx(input_folder, output_folder)

Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-TXT/DCM_11-20-24_VJ_T1 - PowerVJ.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel/DCM_11-20-24_VJ_T1 - PowerVJ.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-TXT/KRH_11-20-24_VJ_T1 - PowerVJ.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel/KRH_11-20-24_VJ_T1 - PowerVJ.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-TXT/JTH_11-20-24_VJ_T2 - PowerVJ.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel/JTH_11-20-24_VJ_T2 - PowerVJ.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-TXT/BCS_11-21-23_VJ_T1 - PowerVJ 2.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel/BCS_11-21-23_VJ_T1 -

In [22]:
import os
import pandas as pd

# Define a function to analyze force data and report specific values including normalized GRF at VEM5
def analyze_force_data(file_path):
    # Load the XLSX file, skipping the first 7 rows to start at the header (which is in row 8)
    df = pd.read_excel(file_path, skiprows=7)

    # Check if required columns are present
    required_columns = ['VEM1', 'VEM2', 'VEM3', 'VEM4', 'VEM5', 'Total_GRF', 'COM_Velocity']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Column {col} not found in {file_path}")

    # Find the body weight (Total GRF in the first row)
    body_weight = abs(df.loc[0, 'Total_GRF'])

    # Find the event markers for the phases
    if df[df['VEM1'] == 1].empty or df[df['VEM2'] == 1].empty:
        raise ValueError(f"Missing VEM1 or VEM2 markers in {file_path}")
    
    if df[df['VEM3'] == 1].empty or df[df['VEM4'] == 1].empty:
        raise ValueError(f"Missing VEM3 or VEM4 markers in {file_path}")
    
    start_idx_1 = df[df['VEM1'] == 1].index[0]  # Start of Phase 1
    end_idx_1 = df[df['VEM2'] == 1].index[0]    # End of Phase 1
    
    start_idx_2 = df[df['VEM3'] == 1].index[0]  # Start of Phase 2
    end_idx_2 = df[df['VEM4'] == 1].index[0]    # End of Phase 2

    # Extract data for each phase and find the most negative (largest magnitude) GRF
    peak_grf_1 = df['Total_GRF'][start_idx_1:end_idx_1 + 1].min()  # Phase 1
    peak_grf_2 = df['Total_GRF'][start_idx_2:end_idx_2 + 1].min()  # Phase 2

    # Normalize the peak GRF values by body weight
    normalized_peak_grf_1 = abs(peak_grf_1 / body_weight)
    normalized_peak_grf_2 = abs(peak_grf_2 / body_weight)

    # Calculate the average normalized peak force for Phase 1
    avg_normalized_peak_force_1 = abs(normalized_peak_grf_1)
    
    # Calculate percent differences for both phases
    percent_difference_1 = 0  # Since it's a single value now
    percent_difference_2 = 0  # Since it's a single value now

    # Find the GRF and COM velocity when VEM5 is 1
    if df[df['VEM5'] == 1].empty:
        raise ValueError(f"Missing VEM5 marker in {file_path}")

    vem5_idx = df[df['VEM5'] == 1].index[0]
    vem5_grf = abs(df.loc[vem5_idx, 'Total_GRF'])
    vem5_com_velocity = df.loc[vem5_idx, 'COM_Velocity']
    
    # Normalize GRF at VEM5 by body weight
    normalized_vem5_grf = abs(vem5_grf / body_weight)

    # Return the results, including the normalized GRF at VEM5
    return (avg_normalized_peak_force_1, normalized_peak_grf_1, normalized_peak_grf_2, percent_difference_1, percent_difference_2, 
            vem5_grf, vem5_com_velocity, normalized_vem5_grf)

# Set up the folder paths for input and output
input_folder_path = '/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Excel'
output_folder_path = '/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Result'

# Ensure the output folder exists
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

results = []

# Loop through all the files in the input folder
for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(input_folder_path, file_name)
        
        try:
            # Analyze the data from the file
            (avg_normalized_peak_force_1, norm_peak_grf_1, norm_peak_grf_2, 
             percent_diff_1, percent_diff_2, vem5_grf, vem5_com_velocity, 
             normalized_vem5_grf) = analyze_force_data(file_path)
        
            # Extract participant initials from file name
            participant_initials = file_name[:3]
        
            # Append the results to the list
            results.append([participant_initials, avg_normalized_peak_force_1, norm_peak_grf_1, norm_peak_grf_2, 
                            percent_diff_1, percent_diff_2, vem5_grf, vem5_com_velocity, normalized_vem5_grf])
        
        except ValueError as e:
            print(f"Error processing file {file_name}: {e}")

# Convert the results into a DataFrame
results_df = pd.DataFrame(results, columns=[
    'Participant', 
    'Avg Normalized Peak Force Phase 1', 
    'Normalized Peak GRF Phase 1', 'Normalized Peak GRF Phase 2', 
    'Percent Difference Phase 1 (%)', 'Percent Difference Phase 2 (%)',
    'GRF at VEM5', 'COM Velocity at VEM5', 'Normalized GRF at VEM5'
])

# Define the output file path
output_file_path = os.path.join(output_folder_path, 'Nov19-22Normalized_Force_Velo_Report.xlsx')

# Save the individual trial results to an Excel file with two sheets in the output folder
with pd.ExcelWriter(output_file_path) as writer:
    # Save individual trials results
    results_df.to_excel(writer, sheet_name='Individual Trials', index=False)
    
    # Group by participant and calculate average metrics
    grouped_results = results_df.groupby('Participant').mean().reset_index()
    
    # Save the averaged results to a separate sheet
    grouped_results.to_excel(writer, sheet_name='Averaged by Participant', index=False)

print(f"Normalized combined report generated and saved to {output_file_path}")


Normalized combined report generated and saved to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJNov19-22-Result/Nov19-22Normalized_Force_Velo_Report.xlsx
